In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from climate import EarthModel
from numpy.random import exponential

In [ ]:
def seconds(years):
    return years * 365.25 * 24 * 3600

### Occlusion response to one eruption

In [ ]:
# Fit curve to eruption data
def fit_eruption(year, value, weights=None):
    if weights is None:
        weights = np.ones_like(value)

    # Translate eruption time to a small number greater than 0
    eps = 0.01
    t = year - year[0] + eps

    # Transform values to look like ~ 1/t
    data = value / value[-1]
    data = 1 / data
    data = data - 1

    # Find a least squares fit to the eruption curve using a 1/t function
    sol = least_squares(lambda x: weights * (x[0]/(t - x[1])**2 - data), [1, 0])

    # Transform to an "occluding" function that reduces incoming radiation
    # for time in seconds since the eruption
    def phi(t):
        # Function was fit in years.
        t = t / 365.25 / 24 / 3600
        return 1 / (sol.x[0]/(t - sol.x[1])**2 + 1)

    return phi

In [ ]:
# Eruption 1 fit
df = pd.read_csv('./eruption_1.csv')
year = df['date'].values
value = df['value'].values
phi_1 = fit_eruption(year, value)
# Shift time to 0 and convert to seconds
t = (year - year[0]) * 365.25 * 24 * 3600
fig, ax = plt.subplots()
ax.plot(t, value / value[-1])
ax.plot(t, phi_1(t))

In [ ]:
# Eruption 2 fit
df = pd.read_csv('./eruption_2.csv')
year = df['date'].values
value = df['value'].values
phi_2 = fit_eruption(year, value)

# Shift time to 0 and convert to seconds
t = (year - year[0]) * 365.25 * 24 * 3600
fig, ax = plt.subplots()
ax.plot(t, value / value[-1])
ax.plot(t, phi_2(t))

In [ ]:
# Take phi as the average of the two fits
phi = lambda t: 0.5 * (phi_1(t)  + phi_2(t))

In [ ]:
# Plot final function
fig, ax = plt.subplots()
years = 5
t = np.linspace(0, years * 365.25 * 24 * 3600)
ax.plot(t, phi(t))
ax.set_xlabel('Time since eruption (s)')
ax.set_ylabel('$\phi(t)$')

### Spatial component of eruption

In [ ]:
model = EarthModel('parameters.json')
model.set_occlusion(phi)
model.max_eruption_time = 100 * 365.25 * 24 * 3600
eruption_time = 1 * 365.25 * 24 * 3600
model.build_eruptions([0, 1, 2, 3, 4, 5], [0, 0, 0, 0, 0, 0])
years = 10
ts = np.linspace(0, years * 365.25 * 24 * 3600, 500)
fig, ax = plt.subplots()
ax.plot(ts, [model.phi(t)[5] for t in ts])
ax.plot(ts, [model.phi(t)[1] for t in ts])
ax.plot(ts, [model.phi(t)[2] for t in ts])
ax.plot(ts, [model.phi(t)[3] for t in ts])
ax.plot(ts, [model.phi(t)[4] for t in ts])
ax.plot(ts, [model.phi(t)[5] for t in ts])

### Time component of multiple eruptions

In [ ]:
df = pd.read_csv('E3WebApp_Eruptions1960.csv')
df = df[df.StartDateYear > 1959]
dates = df['StartDate'].values

#Drop duplicates
df2 = df.drop_duplicates(subset=['VolcanoNumber'])
lat = df2['LatitudeDecimal'].values

#Count latitude occurrences
value_counts = df['LatitudeDecimal'].value_counts(dropna=True, sort=True)
df_value_counts = pd.DataFrame(value_counts)
df_value_counts = df_value_counts.reset_index()
df_value_counts.columns = ['Latitude', 'Counts']

#Find number of volcanoes per zone
def Zones_func(df_value_counts):
    zone1 = []
    zone2 = []
    zone3 = []
    zone4 = []
    zone5 = []
    zone6 = []
    zone1_erc = []
    zone2_erc = []
    zone3_erc = []
    zone4_erc = []
    zone5_erc = []
    zone6_erc = []
    for i in range(len(df_value_counts)):
        if df_value_counts.iloc[i][0] > 60:
            zone6.append(df_value_counts.iloc[i][0])
            zone6_erc.append(df_value_counts.iloc[i][1])
        if df_value_counts.iloc[i][0] > 30 and df_value_counts.iloc[i][0] < 60:
            zone5.append(df_value_counts.iloc[i][0])
            zone5_erc.append(df_value_counts.iloc[i][1])
        if df_value_counts.iloc[i][0] > 0 and df_value_counts.iloc[i][0] <30:
            zone4.append(df_value_counts.iloc[i][0])
            zone4_erc.append(df_value_counts.iloc[i][1])
        if df_value_counts.iloc[i][0] <0 and df_value_counts.iloc[i][0] >-30:
            zone3.append(df_value_counts.iloc[i][0])
            zone3_erc.append(df_value_counts.iloc[i][1])
        if df_value_counts.iloc[i][0] <-30 and df_value_counts.iloc[i][0]>-60:
            zone2.append(df_value_counts.iloc[i][0])
            zone2_erc.append(df_value_counts.iloc[i][1])
        if df_value_counts.iloc[i][0] <-60 and df_value_counts.iloc[i][0]>-90:
            zone1.append(df_value_counts.iloc[i][0])
            zone1_erc.append(df_value_counts.iloc[i][1])
    volc_num = [len(zone1),len(zone2),len(zone3),len(zone4),len(zone5),len(zone6)]
    return (volc_num,zone1,zone2,zone3,zone4,zone5,zone6,zone1_erc,zone2_erc,zone3_erc,zone4_erc,zone5_erc,zone6_erc)

Zones = Zones_func(df_value_counts)
#Number of volcanoes per zone
volc = Zones[0]
#Volcano latitudes
Zone1 = Zones[1]
Zone2 = Zones[2]
Zone3 = Zones[3]
Zone4 = Zones[4]
Zone5 = Zones[5]
Zone6 = Zones[6]
Zone1e = np.sum(Zones[7])
Zone2e = np.sum(Zones[8])
Zone3e = np.sum(Zones[9])
Zone4e = np.sum(Zones[10])
Zone5e = np.sum(Zones[11])
Zone6e = np.sum(Zones[12])

In [ ]:
# Find average repose time per zone (mind we're neglecting the eruption length)
t_tot = (45 + 7/12 + 8/365.25)*365.25*24*3600
t_tot_yr = (45 + 7/12 + 8/365.25)

# Zonal average repose time (in yrs)
Zone_av_yr = []
for i in range(7,13):
    Zone_av_yr.append(np.sum(Zones[i])/t_tot_yr)

# Zonal average repose time (in s)
Zone_av_s = []
for i in range(7,13):
    Zone_av_s.append(np.sum(Zones[i])/t_tot)

In [ ]:
repose_times = np.array(Zone_av_yr) * 365.25 * 24 * 3600
tf = seconds(20)
n_eruptions = tf / repose_times
n_eruptions = n_eruptions.astype(int)

# Add eruptions for each zone spaced equally by the repose time
eruption_zones = []
eruption_times = []
for i in range(6):
    repose_time = repose_times[i]
    n = n_eruptions[i]
    for j in range(n):
        eruption_zones.append(i)
        eruption_times.append(repose_time * j)
n_eruptions

In [ ]:
model = EarthModel('parameters.json')
model.t0 = 0
model.tf = 100 * 3.154e+7 # Years
model.tn = 500
model.max_eruption_time = 100 * 365.25 * 24 * 3600
model.method = 'vode'
model.T0 = [274.12, 279.34, 282.26, 280.88, 279.71, 274.93]
model.set_occlusion(phi)
model.build_eruptions(eruption_zones, eruption_times)
model.build()
model.solve()

# Plot solution
fig, ax = plt.subplots(figsize=(8, 6))
for i in range(model.size):
    ax.plot(model.t, model.T[:, i], label='Zone {}'.format(i))
ax.legend()

In [ ]:
# Add eruptions for each zone through a poisson process based on
# the average repose time
test_repose_times = np.array([100, 50, 20, 20, 50, 100]) * 365.25 * 24 * 3600
np.random.seed(42)
zone_eruption_times = np.array([
    exponential(test_repose_times[i], 200).cumsum() for i in range(6)
])
eruption_zones = []
eruption_times = []
for i in range(6):
    for t in zone_eruption_times[i]:
        eruption_zones.append(i)
        eruption_times.append(t)

In [ ]:
model.build_eruptions(eruption_zones, eruption_times)
years = 100
ts = np.linspace(0, years * 365.25 * 24 * 3600, 500)
phis = [model.phi(t) for t in ts]

In [ ]:
fig, ax = plt.subplots()
ax.plot(ts, [phi[0] for phi in phis])
ax.plot(ts, [phi[1] for phi in phis])
ax.plot(ts, [phi[2] for phi in phis])
ax.plot(ts, [phi[3] for phi in phis])
ax.plot(ts, [phi[4] for phi in phis])
ax.plot(ts, [phi[5] for phi in phis])

In [ ]:
model = EarthModel('parameters.json')
model.t0 = 0
model.tf = 100 * 3.154e+7 # Years
model.tn = 500
model.max_eruption_time = 100 * 365.25 * 24 * 3600
model.method = 'vode'
model.T0 = [274.12, 279.34, 282.26, 280.88, 279.71, 274.93]
model.set_occlusion(phi)
model.build_eruptions(eruption_zones, eruption_times)
model.build()
model.solve()

In [ ]:
# Plot solution
nrows = 2
fig = plt.figure(figsize=(8, 6 * 2))
gs = fig.add_gridspec(nrows=nrows, ncols=1)
ax = fig.add_subplot(gs[0, 0])
for i in range(model.size):
    ax.plot(model.t, model.T[:, i], linewidth=2, label='Zone {}'.format(i), zorder=1)

ax.legend()
ax.set_xlim((0, seconds(100)))
# Plot zone 0 eruption times as vertical lines
for i in range(6):
    for t in zone_eruption_times[i]:
        ax.axvline(
            t, color='C{}'.format(i),
            zorder=0, linewidth=0.7, linestyle='dashed'
        )